In [2]:
from scipy.optimize import minimize
import numpy as np
import voronotalt as voro

%run formfactor.py
%run debyecalc_Iq.py
%run load_testdata.py
%run mapping_aas.py

from load_testdata import read_profile_pepsi, load_xyz

PDB Atom Map: {'OXT': 'O_', 'OT1': 'O_', 'OT2': 'O', 'OCT1': 'O_', 'OCT2': 'O', 'OT': 'O_', 'O1': 'O_', 'O2': 'O', 'C': 'C', 'O': 'O', 'N': 'NH', 'CA': 'CH'}
Atom Map: {'C': 'C', 'O': 'O', 'N': 'N', 'S': 'S', 'P': 'P', 'MG': 'MG', 'IR': 'IR', 'FE': 'FE', 'CU': 'CU', 'ZN': 'ZN', 'CA': 'CA', 'CR': 'Cr', 'CL': 'CL', 'SE': 'SE', 'H': 'H', 'I': 'I', 'IOD': 'I', 'D': 'D'}
PDB Map: {'ALA.CB': 'CH3', 'ARG.CG': 'CH2', 'ARG.CD': 'CH2', 'ARG.CZ': 'C', 'ARG.CB': 'CH2', 'ASN.CB': 'CH2', 'ASN.CG': 'C', 'ASN.OD1': 'O', 'ASN.ND2': 'NH2', 'ASP.CB': 'CH2', 'ASP.CG': 'C', 'CYS.CB': 'CH2', 'CYS.SG': 'SH', 'SEC.CB': 'CH2', 'SEC.SE': 'SEH', 'GLU.CB': 'CH2', 'GLU.CG': 'CH2', 'GLU.CD': 'C', 'GLN.CB': 'CH2', 'GLN.CG': 'CH2', 'GLN.CD': 'C', 'GLN.OE1': 'O', 'GLN.NE2': 'NH2', 'GLY.CA': 'CH2', 'HIS.CB': 'CH2', 'HIS.CG': 'C', 'HIS.ND1': 'N', 'HIS.CD2': 'Csp2H', 'HIS.CE1': 'Csp2H', 'HIS.NE2': 'NH', 'ILE.CB': 'CH', 'ILE.CG1': 'CH2', 'ILE.CG2': 'CH3', 'ILE.CD1': 'CH3', 'ILE.CD': 'CH3', 'LEU.CB': 'CH2', 'LEU.CG': 'CH',

In [3]:
def function_to_optimize(rs):
    
    r = rs[0] +  rs[1]
    return (r - 8) ** 2

initial_guess = [0.0, 0.0]
result = minimize(function_to_optimize, initial_guess, method='Nelder-Mead')
print("Optimal value:", result.x)
print("Function value at optimal point:", result.fun)

Optimal value: [4.00239582 3.99763404]
Function value at optimal point: 8.912719696666353e-10


In [ ]:
def voro_balls(df_struct, amino_code, radius, probe):
    balls = []
    
     #create dict of radii
    #sas_max =  4 * np.pi * 5**2  # Example maximum SAS value for scaling
    """radius_dict =  {
        "ALA": radius[1] + sasa[0]/sas_max * radius[21],
        "ARG": radius[2] + sasa[1]/sas_max * radius[22],
        "ASN": radius[3] + sasa[2]/sas_max * radius[23],
        "ASP": radius[4] + sasa[3]/sas_max * radius[24],
        "CYS": radius[5] + sasa[4]/sas_max * radius[25],
        "GLU": radius[6] + sasa[5]/sas_max * radius[26],
        "GLN": radius[7] + sasa[6]/sas_max * radius[27], 
        "GLY": radius[8] + sasa[7]/sas_max * radius[28],
        "HIS": radius[9] + sasa[8]/sas_max * radius[29],
        "ILE": radius[10] + sasa[9]/sas_max * radius[30],
        "LEU": radius[11] + sasa[10]/sas_max * radius[31],
        "LYS": radius[12] + sasa[11]/sas_max * radius[31],
        "MET": radius[13] + sasa[12]/sas_max * radius[31],
        "PHE": radius[14] + sasa[13]/sas_max * radius[33],
        "PRO": radius[15] + sasa[14]/sas_max * radius[34],
        "SER": radius[16] + sasa[15]/sas_max * radius[35],
        "THR": radius[17] + sasa[16]/sas_max * radius[36],
        "TRP": radius[18] + sasa[17]/sas_max * radius[37],
        "TYR": radius[19] + sasa[18]/sas_max * radius[38],
        "VAL": radius[20] + sasa[19]/sas_max * radius[39],
    }"""
    radius_dict =  {
        "ALA": radius[1] ,
        "ARG": radius[2] ,
        "ASN": radius[3] ,
        "ASP": radius[4] ,
        "CYS": radius[5] ,
        "GLU": radius[6] ,
        "GLN": radius[7] , 
        "GLY": radius[8] ,
        "HIS": radius[9] ,
        "ILE": radius[10] ,
        "LEU": radius[11] ,
        "LYS": radius[12] ,
        "MET": radius[13] ,
        "PHE": radius[14] ,
        "PRO": radius[15] ,
        "SER": radius[16] ,
        "THR": radius[17] ,
        "TRP": radius[18] ,
        "TYR": radius[19] ,
        "VAL": radius[20] ,
    }
    for a in range(len(df_struct)):
        aa = amino_code[a]
        balls.append(voro.Ball(df_struct[a][0], df_struct[a][1], df_struct[a][2], radius_dict[aa]))
        #print(f"Atom {a}: {res_code[a]} {amino_code[a]} mapped to {aa} with radius {params.fj[aa].r}")

    rt = voro.RadicalTessellation(balls, probe)
    cells=rt.cells
    
    vols = [cell.volume for cell in cells]
    sasa = [cell.sas_area for cell in cells]
    #print([cell.sas_area for cell in cells])
    #total_vol = sum(vols)   
    return vols, sasa


def optimize_vols(radius, volumes, amino, coords):
    probe = radius[0]  # get the probe radius from the list
    
     # Initialize sasa with zeros or appropriate values
    all_voro = np.zeros(len(coords))
    for i in range(len(coords)):
        voro, sasa = voro_balls(coords[i], amino[i], radius, probe)
        all_voro[i] = sum(abs(volumes[i] - voro))
    return all_voro.sum()

def optimize_list(radius, tot_volume, amino, coords):
    probe = radius[0]  # get the probe radius from the list
    
     # Initialize sasa with zeros or appropriate values
    #sasa_list = np.zeros(len(coords))
    #for i in range(len(coords)):
    #    voro, sasa_list[i] = voro_balls(coords[i], amino[i], [0] * len(amino[i]), radius, probe)
    
    return sum((tot_volume[i] - voro_balls(coords[i], amino[i], radius, probe)[0]) for i in range(len(coords)))

In [ ]:
amino = []
coords = []
list = ["../cg_structures/SASDAC2.xyz", "../cg_structures/SASDAH2.xyz", "../cg_structures/SASDAJ6.xyz", "../cg_structures/SASDAQ2.xyz", "../cg_structures/SASDAW3.xyz"]
for file in list:  
    amino_part, coords_part = load_xyz(file)
    amino.append(amino_part)
    coords.append(coords_part)
print(len(amino), len(coords))

tot_volume = []
pepsi_files = ["pepsi-saxs_data/SASDAC2_fit1_model1.out", "pepsi-saxs_data/SASDAH2_fit1_model1.out", "pepsi-saxs_data/SASDAJ6_fit1_model1.out", "pepsi-saxs_data/SASDAQ2_fit1_model1.out", "pepsi-saxs_data/SASDAW3_fit1_model1.out"]
for pepsi in pepsi_files:
    Iq_tot, Iat, Iev, Ihs, atev, aths, evhs, q_pepsi, pepsi_scaling, volume = read_profile_pepsi(pepsi, pepsi.replace('.out', '.log'))
    tot_volume.append(volume)
    
print(len(tot_volume))

In [ ]:
n_params =  20   # amino acid radii x 2 
radius = np.array([1., 3.1, 4.0, 3.6, 3.6, 3.6,  3.8, 3.8, 2.9, 3.9, 3.6,
                   3.6, 3.7,3.8, 3.9, 3.4, 3.3, 3.5, 4.3, 4.1, 3.4])  # initial guess for probe radius + amino acid radii

bounds = [(0, 2)] + [(0, radius[i] +1) for i in range(1,len(radius))]
print(bounds)
result = minimize(optimize_list, radius, args=(tot_volume, amino, coords), bounds=bounds, tol = 1, method= "Powell")

In [ ]:
radius = np.zeros(21)
radius_dict =  {
        "ALA": radius[1],
        "ARG": radius[2],
        "ASN": radius[3],
        "ASP": radius[4],
        "CYS": radius[5],
        "GLU": radius[6],
        "GLN": radius[7],
        "GLY": radius[8],
        "HIS": radius[9],
        "ILE": radius[10],
        "LEU": radius[11],
        "LYS": radius[12],
        "MET": radius[13],
        "PHE": radius[14],
        "PRO": radius[15],
        "SER": radius[16],
        "THR": radius[17],
        "TRP": radius[18],
        "TYR": radius[19],
        "VAL": radius[20],
}

radius_dict["ALA"]

In [ ]:
(result.x)

In [ ]:
result.fun


In [ ]:
exp_data = '../../experimental_data/SASDAH2_fit1_model1.pdb'
def extract_volumes(pdb_file):
    params = cSAXSparameters()
    atoms = []
    res_names = []
    seq = []
    with open(pdb_file, 'r') as file:
        for line in file:
            if line.startswith("ATOM"):
            # Extract x, y, z coordinates
            # PDB format: ATOM | serial | name | res_name | chain | res_seq | x | y | z | ...
                parts = line.split()
                atoms.append(parts[2])  # Atom name
                res_names.append(parts[3])
                seq.append(parts[5])
    tot_vol = np.zeros((int(seq[-1])))
    counter = 0
    amino_acids = []
    for a, atom in enumerate(atoms):
        if atom in pdb_atom_map.mapping:
            aa = pdb_atom_map.mapping[atom]       
        else:
            aa = res_names[a] + "." + atom
            aa = pdb_map.mapping[aa]
        vol = params.fj[aa].dsv     
        if a == 0:
            tot_vol[0] += vol
        elif seq[a] == seq[a-1]:
            tot_vol[counter] += vol
        else:
            amino_acids.append(res_names[a-1])
            counter += 1
            tot_vol[counter] += vol          
    return tot_vol, amino_acids
vol, aas = extract_volumes(exp_data)

In [ ]:

vol

In [ ]:
aas.index('ALA')


In [ ]:
vol_dict = {}
for i, aa in enumerate(aas):
    if aa not in vol_dict:
        vol_dict[aa] = vol[i]

In [ ]:
vol_dict


In [ ]:
vol, aas = extract_volumes('../../experimental_data/SASDAC2_fit1_model1.pdb')
for i, aa in enumerate(aas):
    if aa not in vol_dict:
        vol_dict[aa] = vol[i]

In [ ]:
vol_dict
r_dict = vol_dict.copy()
for key in r_dict:
    r_dict[key] = (3 * vol_dict[key] / (4 * np.pi)) ** (1/3)

In [4]:
r_dict

NameError: name 'r_dict' is not defined